In [1]:
import pandas as pd
from pandas import read_csv
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from numpy import argmax
import numpy as np

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# start with linear classifiers, non-linear ones and eventually the more complex neural nets

In [2]:
# load a single file as a numpy array
# def load_file(filepath):
#     dataframe = read_csv(filepath, header=None, delim_whitespace=True)
#     return dataframe.values

# load a dataset group, such as train or test
# def load_dataset_group(group, prefix=''):
#     # load input data
#     X = load_file(prefix + group + '/X_'+group+'.txt')
#     # load class output
#     y = load_file(prefix + group + '/y_'+group+'.txt')
#     return X, y

# load the dataset, returns train and test X and y elements
# def load_dataset(prefix=''):
#     # load all train
#     trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

# def load_dataset(prefix=''):
#     # load all train
#     train = load_dataset_group('train', prefix + 'HARDataset/')
#     print(trainX.shape, trainy.shape)
#     # load all test
#     testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
#     print(testX.shape, testy.shape)
#     # flatten y
#     trainy, testy = trainy[:,0], testy[:,0]
#     print(trainX.shape, trainy.shape, testX.shape, testy.shape)
#     return trainX, trainy, testX, testy

In [3]:
# path = '../data/processed/flori5hz.csv'
# values = load_file(path)
# #X, y = load_dataset_group()
# values

In [2]:
train_df = pd.read_csv('../data/processed/train.csv')

test_df = pd.read_csv('../data/processed/test.csv')

In [3]:
X_train = train_df.drop(['state', 'name'], axis=1)
y_train = pd.DataFrame(train_df['state'])

X_test = test_df.drop(['state', 'name'], axis=1)
y_test = pd.DataFrame(test_df['state'])

print(y_test.shape)
print(test_df['state'].shape)

(1207, 1)
(1207,)


In [4]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
    # nonlinear models
#     models['knn'] = KNeighborsClassifier()
#     models['cart'] = DecisionTreeClassifier()
#    models['cart'] = { 'clf': DecisionTreeClassifier(), 'params': {}}
    #models['svm'] = SVC()
#     models['bayes'] = GaussianNB()
    # ensemble models
#     models['bag'] = BaggingClassifier(n_estimators=100)
    models['rf'] = RandomForestClassifier(n_estimators=100)
#     models['et'] = ExtraTreesClassifier(n_estimators=100)
    models['gbm'] = GradientBoostingClassifier(n_estimators=100)
#     models['bag'] = BaggingClassifier()
    #models['rf'] = { 'clf': RandomForestClassifier(n_jobs=-1), 'params': {'n_estimators': [100],
#                                                                            'bootstrap': [False],
#                                                                            'min_samples_split': [10, 20, 40, 50],
#                                                                            'max_features': ['log2', 'sqrt','auto', None], 
#                                                                           'criterion': ['entropy', 'gini']}}
#     models['et'] = ExtraTreesClassifier()
#     models['gbm'] = GradientBoostingClassifier()
    # sgd is sensitive to feature scaling
    #models['sgd'] = SGDClassifier()
    #models['gp'] = GaussianProcessClassifier()
#     models['mlp'] = MLPClassifier()
    print('Defined %d models' % len(models))
    return models

In [5]:
# get model list
models = define_models()

Defined 2 models


In [6]:
# evaluate a single model
def evaluate_model(trainX, trainy, testX, testy, model):
    # fit the model
    model.fit(trainX, trainy)
    # make predictions
    yhat = model.predict(testX)
    # evaluate predictions
    accuracy = balanced_accuracy_score(testy, yhat)
    #roc_auc = roc_auc_score(testy, yhat)
    
    print(classification_report(testy, yhat))
    print(confusion_matrix(testy, yhat))
    return accuracy * 100.0
    #return roc_auc * 100.0

# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(trainX, trainy, testX, testy, models):
    results = dict()
    for name, model in models.items():
        # evaluate the model
        results[name] = evaluate_model(trainX, trainy, testX, testy, model)
        # show process
        print('>%s: %.3f' % (name, results[name]))
    return results

# print and plot the results
def summarize_results(results, maximize=True):
    # create a list of (name, mean(scores)) tuples
    mean_scores = [(k,v) for k,v in results.items()]
    # sort tuples by mean score
    mean_scores = sorted(mean_scores, key=lambda x: x[1])
    # reverse for descending order (e.g. for accuracy)
    if maximize:
        mean_scores = list(reversed(mean_scores))
    print()
    for name, score in mean_scores:
        print('Name=%s, Score=%.3f' % (name, score))

In [7]:
# evaluate models
results = evaluate_models(X_train, y_train, X_test, y_test, models)

/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


              precision    recall  f1-score   support

           0       0.94      0.96      0.95       510
           1       0.87      0.96      0.92       533
           2       0.73      0.58      0.65        79
           3       0.89      0.36      0.52        85

    accuracy                           0.90      1207
   macro avg       0.86      0.72      0.76      1207
weighted avg       0.89      0.90      0.89      1207

[[490  12   4   4]
 [ 16 514   3   0]
 [ 12  21  46   0]
 [  3  41  10  31]]
>rf: 71.803


/home/elizabeth/education/chair_sensor/ml_project/venv/lib/python3.5/site-packages/sklearn/ensemble/gradient_boosting.py:1450: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


              precision    recall  f1-score   support

           0       0.95      0.93      0.94       510
           1       0.86      0.97      0.91       533
           2       0.69      0.59      0.64        79
           3       0.83      0.41      0.55        85

    accuracy                           0.89      1207
   macro avg       0.83      0.73      0.76      1207
weighted avg       0.88      0.89      0.88      1207

[[472  23   9   6]
 [ 13 516   4   0]
 [ 12  19  47   1]
 [  1  41   8  35]]
>gbm: 72.507


In [105]:
### summarize results
summarize_results(results)


Name=rf, Score=68.030


In [8]:
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from pandas import DataFrame
import numpy as np

all_df = train_df.append(test_df)
all_df.reset_index(drop=True, inplace=True)

X_all = all_df.drop(['state', 'name'], axis=1)
y_all = all_df['state']

groups = all_df['name']

scorer = make_scorer(balanced_accuracy_score)

outcomes = []

def run_nested_logo(clf, params, scorer):
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_all, groups=groups):
        group += 1
        X_train, X_test = X_all.iloc[train_index], X_all.iloc[test_index]
        y_train, y_test = y_all.iloc[train_index], y_all.iloc[test_index]
        
        inner_groups = all_df.iloc[all_df.index.isin(train_index)]['name']

        logo = LeaveOneGroupOut()
        grid_obj = GridSearchCV(clf, params, scoring=scorer, cv=logo.split(X_train, groups=inner_groups), n_jobs=-1)
        grid_obj = grid_obj.fit(X_train, y_train)
        
        print(grid_obj.best_estimator_)
        clf = grid_obj.best_estimator_

        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        
        accuracy = balanced_accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome))
    
def evaluate_tuned_models(models, scorer):
    results = dict()
    for name, model in models.items():
        print('>%s' % (name))
        clf, params = model['clf'], model['params']
        run_nested_logo(clf, params, scorer)

evaluate_tuned_models(models, scorer)


>rf


TypeError: list indices must be integers or slices, not str

In [21]:
# Random Forest

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(X_train, y_train)
# y_pred = random_forest.predict(X_test)
# y_pred
# acc_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)

# cnt = 0
# sitting = 0
# standing = 0
# for i in range(0, len(y_pred)):
#     if not y_pred[i] == y_test[i]:
#         cnt += 1
#         if y_pred[i] == 1:
#             sitting += 1
#         else:
#             standing += 1
#         print('predicted', y_pred[i], 'actual', y_test[i])
        
# print('\nTotal:', len(y_test))
# print('Wrong:', cnt)
# print('Accuracy:', acc_random_forest)
# print('False positive:', sitting)
# print('False negative:', standing)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneGroupOut

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

groups = train_df['name']



logo = LeaveOneGroupOut()

# Choose the type of classifier. 
clf = RandomForestClassifier()

# clf = GradientBoostingClassifier()

depths = [2, 4, 8, 16, 32, 64, 80, 100]

# Choose some parameter combinations to try
# parameters = {'n_estimators': [100, 200],
#               'bootstrap': [False],
#               'min_samples_split': [10, 20, 40, 50],
#                'max_features': ['log2', 'sqrt','auto', None], 
#                'criterion': ['entropy', 'gini']
# #               'max_depth': depths
#              }

parameters = {'n_estimators': [100]}

# Type of scoring used to compare parameter combinations
acc_scorer = make_scorer(balanced_accuracy_score)

# Run the grid search
grid_obj = GridSearchCV(clf, parameters, scoring=acc_scorer, cv=logo.split(X_train, groups=groups), n_jobs=-1)
grid_obj = grid_obj.fit(X_train, y_train)

# Set the clf to the best combination of parameters
clf = grid_obj.best_estimator_

# Fit the best algorithm to the data. 
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
print(balanced_accuracy_score(y_test, predictions))

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

In [11]:
grid_obj.best_params_

{'n_estimators': 100}

In [53]:
grid_obj.best_params_

{'bootstrap': False,
 'criterion': 'gini',
 'min_samples_split': 10,
 'n_estimators': 200,
 'warm_start': False}

In [43]:
grid_obj.best_estimator_

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [21]:
from sklearn.model_selection import LeaveOneGroupOut
from pandas import DataFrame
import numpy as np

X_train = train_df.drop(['state', 'name'], axis=1)
y_train = train_df['state']
X_test = test_df.drop(['state', 'name'], axis=1)
y_test = test_df['state']

X_all = X_train.append(X_test)
y_all = y_train.append(y_test)

groups = train_df.append(test_df)['name']

outcomes = []

clf = RandomForestClassifier(criterion='gini',
                             max_features=None,
                             min_samples_split=5,
                             n_estimators=100)

clf = GradientBoostingClassifier(n_estimators=100)

def run_logo(clf):
    logo = LeaveOneGroupOut()
    group = 0
    
    for train_index, test_index in logo.split(X_all, groups=groups):
        group += 1
        X_train, X_test = X_all.values[train_index], X_all.values[test_index]
        y_train, y_test = y_all.values[train_index], y_all.values[test_index]
        clf.fit(X_train, y_train)
        predictions = clf.predict(X_test)
        accuracy = balanced_accuracy_score(y_test, predictions)
        outcomes.append(accuracy)
        print("Group {0} accuracy: {1}".format(group, accuracy))
    mean_outcome = np.mean(outcomes)
    print("Mean Accuracy: {0}".format(mean_outcome)) 

run_logo(clf)


Group 1 accuracy: 0.74502341311965
Group 2 accuracy: 0.7084465932863271
Group 3 accuracy: 0.4614235167312559
Group 4 accuracy: 0.5555467040971589
Group 5 accuracy: 0.47061931938279433
Group 6 accuracy: 0.8153399197516844
Group 7 accuracy: 0.722132992651308
Group 8 accuracy: 0.7543498168498168
Mean Accuracy: 0.6541102844837494


In [1]:
# testing keras neural network models
from numpy import mean
from numpy import std
from numpy import dstack
import numpy as np
from pandas import read_csv
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot


from sklearn.metrics import classification_report

Using TensorFlow backend.


In [6]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = read_csv(filepath, header=None)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
    # stack group so that features are the 3rd dimension
    print('loaded', len(loaded[0]))
    print('loaded', loaded)
    loaded = dstack(loaded)
    print('stacked', loaded.shape)
    print('stacked', loaded)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, freq, win, prefix=''):
    filepath = prefix + group + '/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'total_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body acceleration
#     filenames += ['body_acc_x_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_y_'+group+'_'+win+'_'+freq+'.csv', 'body_acc_z_'+group+'_'+win+'_'+freq+'.csv']
#     # body gyroscope
#     filenames += ['body_gyro_x_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_y_'+group+'_'+win+'_'+freq+'.csv', 'body_gyro_z_'+group+'_'+win+'_'+freq+'.csv']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/state_'+group+'_'+win+'_'+freq+'.csv')
    print('X:', filenames)
    print('y:', prefix + group + '/state_'+group+'_'+win+'_'+freq+'.csv')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(freq, win, prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', freq, win, prefix)
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test', freq, win, prefix)
    print(testX.shape, testy.shape)
    # zero-offset class values
#     trainy = trainy - 1
#     testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    y_true = testy
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy, y_true

# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [5]:
# lstm model
# from sklearn.metrics import confusion_matrix
from keras.layers import Bidirectional

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 20, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    # fit network
#     model.fit(trainX, trainy, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=verbose)
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    return prediction, accuracy, model

# run an experiment
def run_experiment(freq, win, repeats=1):
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
#     score = evaluate_model(trainX, trainy, testX, testy)
#     print(score)
    # repeat experiment
    scores = list()
#     for r in range(repeats):
    pred_classes, score, model = evaluate_model(trainX, trainy, testX, testy)
    print(score)
    score = score * 100.0
    print('>#%d: %.3f' % (1, score))
    scores.append(score)
#     print(classification_report(y_true, pred_classes))
#     print(confusion_matrix(y_true, pred_classes))
    # summarize results
    summarize_results(scores)
    return pred_classes, y_true, model

# run the experiment
freq = '50'
win = '50'
pred_classes, y_true, model = run_experiment(freq, win)

loaded 26349
loaded [array([[-0.02734375, -0.03125   , -0.03808594, ..., -0.0234375 ,
        -0.03125   , -0.02148438],
       [-0.03222656, -0.03320312, -0.03125   , ..., -0.02636719,
        -0.03027344, -0.02929688],
       [-0.02050781, -0.02734375, -0.03222656, ..., -0.03125   ,
        -0.0390625 , -0.02539062],
       ...,
       [ 0.07519531,  0.07324219,  0.07128906, ...,  0.06738281,
         0.07519531,  0.07421875],
       [ 0.0703125 ,  0.07910156,  0.0703125 , ...,  0.07324219,
         0.07714844,  0.08203125],
       [ 0.07519531,  0.07226562,  0.07714844, ...,  0.07519531,
         0.07324219,  0.07226562]]), array([[-0.11816406, -0.11523438, -0.11230469, ..., -0.11914062,
        -0.1171875 , -0.12109375],
       [-0.1171875 , -0.12109375, -0.11523438, ..., -0.125     ,
        -0.11816406, -0.11425781],
       [-0.11621094, -0.12011719, -0.11425781, ..., -0.12207031,
        -0.12402344, -0.11914062],
       ...,
       [ 0.0078125 ,  0.00195312,  0.        , ...,  

26349/26349 [==============================] - 58s 2ms/step - loss: 0.6073 - categorical_accuracy: 0.7817
Epoch 4/20
26349/26349 [==============================] - 56s 2ms/step - loss: 0.4679 - categorical_accuracy: 0.8255
Epoch 5/20
26349/26349 [==============================] - 61s 2ms/step - loss: 0.4153 - categorical_accuracy: 0.8522
Epoch 6/20
26349/26349 [==============================] - 56s 2ms/step - loss: 0.3901 - categorical_accuracy: 0.8630
Epoch 7/20
26349/26349 [==============================] - 54s 2ms/step - loss: 0.3650 - categorical_accuracy: 0.8723
Epoch 8/20
26349/26349 [==============================] - 63s 2ms/step - loss: 0.3601 - categorical_accuracy: 0.8721
Epoch 9/20
26349/26349 [==============================] - 58s 2ms/step - loss: 0.3410 - categorical_accuracy: 0.8802
Epoch 10/20
26349/26349 [==============================] - 62s 2ms/step - loss: 0.3284 - categorical_accuracy: 0.8838
Epoch 11/20
26349/26349 [==============================] - 62s 2ms/step - 

In [8]:
freq='50'
win='50'
trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')


loaded 4288
loaded [array([[ 0.00683594,  0.00195312,  0.00878906, ...,  0.00976562,
         0.01074219,  0.01074219],
       [ 0.00097656,  0.0078125 ,  0.00292969, ...,  0.00683594,
         0.        ,  0.00585938],
       [ 0.00585938, -0.00390625,  0.00585938, ...,  0.00292969,
         0.01074219,  0.01074219],
       ...,
       [ 0.07324219,  0.06152344,  0.06640625, ...,  0.06835938,
         0.07519531,  0.07519531],
       [ 0.07714844,  0.07421875,  0.07714844, ...,  0.0703125 ,
         0.07421875,  0.07324219],
       [ 0.07714844,  0.07226562,  0.06835938, ...,  0.06835938,
         0.08007812,  0.07226562]]), array([[-0.08496094, -0.08300781, -0.08300781, ..., -0.07226562,
        -0.07519531, -0.08984375],
       [-0.07421875, -0.07714844, -0.07226562, ..., -0.09179688,
        -0.08886719, -0.08789062],
       [-0.078125  , -0.0859375 , -0.08691406, ..., -0.08496094,
        -0.078125  , -0.08105469],
       ...,
       [ 0.01464844, -0.00976562, -0.00878906, ...,  0

In [26]:
sample = [[ [ 0.0078125, -0.015625, -1.013671875 ],[ -0.00390625, -0.017578125, -1.02734375 ],[ 0.0087890625, -0.0078125, -1.015625 ],[ 0.0009765625, -0.009765625, -1.0234375 ],  [ 0.009765625, -0.00390625, -1.0224609375 ],  [ -0.001953125, -0.01171875, -1.013671875 ],  [ -0.00390625, -0.013671875, -1.013671875 ],  [ -0.0009765625, -0.00390625, -1.0166015625 ],  [ -0.0009765625, -0.0078125, -1.013671875 ],  [ 0.0078125, 0, -1.01171875 ],  [ -0.0029296875, 0, -1.0302734375 ],  [ 0, 0.0029296875, -1.0078125 ],  [ 0.001953125, -0.005859375, -1.009765625 ],  [ -0.0009765625, 0.00390625, -1.0205078125 ],  [ 0, -0.0146484375, -1.009765625 ],  [ -0.0009765625, -0.0078125, -1.0029296875 ],  [ 0.0078125, -0.005859375, -1.0283203125 ],  [ 0.0029296875, -0.0107421875, -1.0107421875 ],  [ 0.0029296875, -0.01171875, -1.0126953125 ],  [ -0.0009765625, -0.0126953125, -1.0068359375 ],  [ 0.0048828125, -0.0087890625, -1.0166015625 ],  [ 0.001953125, -0.0048828125, -1.0185546875 ],  [ 0.0029296875, -0.00390625, -1.0029296875 ],  [ 0.0009765625, -0.00390625, -1.015625 ],  [ -0.00390625, -0.0087890625, -1.017578125 ],  [ 0.001953125, -0.013671875, -1.0166015625 ],  [ 0.0087890625, -0.0078125, -1.013671875 ],  [ -0.0087890625, -0.0087890625, -1.0283203125 ],  [ 0.0009765625, -0.0068359375, -1.0244140625 ],  [ -0.0029296875, -0.00390625, -1.0322265625 ],  [ 0.0029296875, -0.013671875, -1.0341796875 ],  [ -0.00390625, -0.0126953125, -1.021484375 ],  [ 0.005859375, -0.0068359375, -1.0283203125 ],  [ 0.00390625, -0.0078125, -1.0078125 ],  [ -0.0009765625, -0.0068359375, -1.0302734375 ],  [ 0.0009765625, -0.0068359375, -1.0224609375 ],  [ -0.001953125, -0.0068359375, -1.017578125 ],  [ 0, -0.009765625, -1.0068359375 ],  [ -0.001953125, -0.0146484375, -1.01953125 ],  [ 0.005859375, -0.0107421875, -1.0234375 ],  [ 0, 0.001953125, -1.0302734375 ],  [ -0.005859375, -0.0068359375, -1.00390625 ],  [ -0.0048828125, -0.0078125, -1.025390625 ],  [ -0.0009765625, -0.00390625, -1.009765625 ],  [ 0.0078125, -0.0107421875, -1.02734375 ],  [ 0.001953125, -0.0146484375, -1.0078125 ],  [ 0.001953125, -0.0078125, -1.0234375 ],  [ -0.001953125, -0.0087890625, -1.017578125 ],  [ 0.00390625, -0.0068359375, -1.017578125 ],  [ 0, -0.009765625, -1.01953125 ]]]
print(sample[0])

trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
probs = model.predict_proba(testX)

print(probs)
print(testX[0])

[[0.0078125, -0.015625, -1.013671875], [-0.00390625, -0.017578125, -1.02734375], [0.0087890625, -0.0078125, -1.015625], [0.0009765625, -0.009765625, -1.0234375], [0.009765625, -0.00390625, -1.0224609375], [-0.001953125, -0.01171875, -1.013671875], [-0.00390625, -0.013671875, -1.013671875], [-0.0009765625, -0.00390625, -1.0166015625], [-0.0009765625, -0.0078125, -1.013671875], [0.0078125, 0, -1.01171875], [-0.0029296875, 0, -1.0302734375], [0, 0.0029296875, -1.0078125], [0.001953125, -0.005859375, -1.009765625], [-0.0009765625, 0.00390625, -1.0205078125], [0, -0.0146484375, -1.009765625], [-0.0009765625, -0.0078125, -1.0029296875], [0.0078125, -0.005859375, -1.0283203125], [0.0029296875, -0.0107421875, -1.0107421875], [0.0029296875, -0.01171875, -1.0126953125], [-0.0009765625, -0.0126953125, -1.0068359375], [0.0048828125, -0.0087890625, -1.0166015625], [0.001953125, -0.0048828125, -1.0185546875], [0.0029296875, -0.00390625, -1.0029296875], [0.0009765625, -0.00390625, -1.015625], [-0.003

In [16]:
##### from keras.models import model_from_json
import os

trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')

# Saving model
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# Loading model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
#loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

_, accuracy = model.evaluate(testX, testy, batch_size=1, verbose=1)
print('>#%d: %.3f' % (1, accuracy))

loaded 24481
stacked (24481, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(24481, 50, 3) (24481, 1)
loaded 2985
stacked (2985, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(2985, 50, 3) (2985, 1)
(24481, 50, 3) (24481, 4) (2985, 50, 3) (2985, 4)
Saved model to disk
Loaded model from disk
2985/2985 [==============================] - 15s 5ms/step
>#1: 0.901


In [6]:
# save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [5]:
from keras.models import load_model
model = load_model('model.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


In [13]:
print(y_true)

[[1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [2]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [3]
 [3]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]


In [12]:
freq='50'
win='50'
trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')

model.predict_classes(testX)

loaded 27474
loaded [array([[-0.00585938, -0.0078125 ,  0.00390625, ...,  0.01367188,
         0.01855469,  0.01367188],
       [ 0.00097656, -0.00878906, -0.00292969, ...,  0.01855469,
         0.00976562,  0.0078125 ],
       [-0.02246094, -0.01953125, -0.02246094, ...,  0.0234375 ,
         0.0078125 ,  0.00976562],
       ...,
       [-0.015625  , -0.00683594, -0.01367188, ..., -0.01464844,
        -0.00976562, -0.01269531],
       [-0.00683594, -0.00976562, -0.0078125 , ..., -0.01074219,
        -0.00976562, -0.00976562],
       [-0.01660156, -0.01074219, -0.00683594, ..., -0.00585938,
        -0.00097656, -0.015625  ]]), array([[-0.09472656, -0.08496094, -0.08984375, ..., -0.08105469,
        -0.07128906, -0.06933594],
       [-0.08496094, -0.09179688, -0.09082031, ..., -0.08007812,
        -0.08984375, -0.08496094],
       [-0.08886719, -0.08886719, -0.09082031, ..., -0.07421875,
        -0.07910156, -0.07324219],
       ...,
       [-0.11328125, -0.10253906, -0.10839844, ..., -

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [19]:
example = []
for i in range(0,50):
    example.append([1,0,-0.5])
# print([example][0])
model.predict_proba([[example]])


array([[2.3814168e-02, 3.8795219e-15, 5.5544578e-18, 9.7618586e-01]],
      dtype=float32)

In [13]:
trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
print(testX[0])

loaded 24481
stacked (24481, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(24481, 50, 3) (24481, 1)
loaded 2985
stacked (2985, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(2985, 50, 3) (2985, 1)
(24481, 50, 3) (24481, 4) (2985, 50, 3) (2985, 4)
[[ 8.20312500e-02 -4.58984375e-02  9.87304688e-01]
 [ 8.59375000e-02 -5.56640625e-02  1.00976562e+00]
 [ 8.39843750e-02 -4.29687500e-02  9.99023438e-01]
 [ 8.59375000e-02 -4.49218750e-02  9.96093750e-01]
 [ 8.49609375e-02 -4.68750000e-02  1.00292969e+00]
 [ 8.69140625e-02 -3.61328125e-02  9.94140625e-01]
 [ 8.20312500e-02 -4.39453125e-02  9.91210938e-01]
 [ 9.17968750e-02 -3.71093750e-02  9.96093750e-01]
 [ 8.30078125e-02 -3.80859375e-02  1.00097656e+00]
 [ 8.98437500e-02 -5.07812500e-02  9.91210938e-01]
 [ 8.69140625e-02 -3.41796875e-02

In [1]:
# # Save model in tfjs layer format
# import tensorflowjs as tfjs

# tfjs.converters.save_keras_model(model, "./")

ImportError: No module named 'tensorflowjs'

In [63]:
# cnn-lstm model

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from matplotlib import pyplot

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 1, 15, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    
    #splitting windows into blocks because cnn learns features from the blocks,
    #and the window timesteps have dependencies we want to learn
    # reshape data into time steps of sub-sequences
    n_steps, n_length = 1, 50
    trainX = trainX.reshape((trainX.shape[0], n_steps, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, n_length, n_features))
    # define model
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None,n_length,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu')))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(100))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    return prediction, accuracy

# run an experiment
def run_experiment(freq, win, repeats=1):
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
#     score = evaluate_model(trainX, trainy, testX, testy)
#     print(score)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        pred_classes, score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
        print(classification_report(y_true, pred_classes))
        print(confusion_matrix(y_true, pred_classes))
    # summarize results
    summarize_results(scores)

# run the experiment
freq = '50'
win = '50'
run_experiment(freq, win)

loaded 22952
stacked (22952, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(22952, 50, 3) (22952, 1)
loaded 4514
stacked (4514, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(4514, 50, 3) (4514, 1)
(22952, 50, 3) (22952, 4) (4514, 50, 3) (4514, 4)
Epoch 1/15
22952/22952 [==============================] - 44s 2ms/step - loss: 0.6862 - categorical_accuracy: 0.7197
Epoch 2/15
22952/22952 [==============================] - 42s 2ms/step - loss: 0.3934 - categorical_accuracy: 0.8586
Epoch 3/15
22952/22952 [==============================] - 42s 2ms/step - loss: 0.3613 - categorical_accuracy: 0.8688
Epoch 4/15
22952/22952 [==============================] - 42s 2ms/step - loss: 0.3318 - categorical_accuracy: 0.8788
Epoch 5/15
22952/22952 [==============================] - 42s 2ms/step - lo

In [56]:
# convlstm model
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    # define model
    verbose, epochs, batch_size = 1, 15, 10
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    # reshape into subsequences (samples, time steps, rows, cols, channels)
    n_steps, n_length = 1, 50
    trainX = trainX.reshape((trainX.shape[0], n_steps, 1, n_length, n_features))
    testX = testX.reshape((testX.shape[0], n_steps, 1, n_length, n_features))
#     print('reshaped')
#     print(trainX.shape)
    # define model
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,3), activation='relu', input_shape=(n_steps, 1, n_length, n_features)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=1)
    prediction = model.predict_classes(testX)
    return prediction, accuracy

# run an experiment
def run_experiment(freq, win, repeats=1):
    # load data
    trainX, trainy, testX, testy, y_true = load_dataset(freq=freq, win=win, prefix='../data/processed/')
#     score = evaluate_model(trainX, trainy, testX, testy)
#     print(score)
    # repeat experiment
    scores = list()
    for r in range(repeats):
        pred_classes, score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
        print(classification_report(y_true, pred_classes))
        print(confusion_matrix(y_true, pred_classes))
    # summarize results
    summarize_results(scores)

# run the experiment
freq = '50'
win = '50'
run_experiment(freq, win)

loaded 9182
stacked (9182, 50, 3)
X: ['total_acc_x_train_50_50.csv', 'total_acc_y_train_50_50.csv', 'total_acc_z_train_50_50.csv']
y: ../data/processed/train/state_train_50_50.csv
(9182, 50, 3) (9182, 1)
loaded 1806
stacked (1806, 50, 3)
X: ['total_acc_x_test_50_50.csv', 'total_acc_y_test_50_50.csv', 'total_acc_z_test_50_50.csv']
y: ../data/processed/test/state_test_50_50.csv
(1806, 50, 3) (1806, 1)
(9182, 50, 3) (9182, 4) (1806, 50, 3) (1806, 4)
Epoch 1/15
9182/9182 [==============================] - 11s 1ms/step - loss: 1.1147 - categorical_accuracy: 0.4720
Epoch 2/15
9182/9182 [==============================] - 10s 1ms/step - loss: 0.7306 - categorical_accuracy: 0.7487
Epoch 3/15
9182/9182 [==============================] - 10s 1ms/step - loss: 0.5211 - categorical_accuracy: 0.8093
Epoch 4/15
9182/9182 [==============================] - 10s 1ms/step - loss: 0.4582 - categorical_accuracy: 0.8404
Epoch 5/15
9182/9182 [==============================] - 10s 1ms/step - loss: 0.4269 - cat